<a href="https://colab.research.google.com/github/ehadsaloF/NewYork-Accident-Analysis/blob/main/Cleaning_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

In [ ]:
#importing data from Maven Analytics
df = pd.read_csv("https://maven-datasets.s3.amazonaws.com/NYC+Traffic+Accidents/NYC+Accidents+2020.csv.zip")

In [ ]:
#checking the datatype of the columns
df.dtypes

In [ ]:
#Changing the datatype of Crash Date and Crash Time 
df["CRASH DATE"] = pd.to_datetime(df['CRASH DATE'], format='%Y-%m-%d')
df["CRASH TIME"] = pd.to_datetime(df['CRASH TIME'])

#confirming the changes
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
# checking if the location, latittude ans longitude have the same records 
#with missing values
df[df['LATITUDE'].isnull()].isna().sum()

In [ ]:
#dropping records with missing latitude and longitude
df = df.dropna(how='all', subset=["LATITUDE"])

df[df['LATITUDE'].isnull()].isna().sum()

In [ ]:
#getting the list of latitude and longitude for missing Borough values
joined_column = df[df['BOROUGH'].isnull()& df['LATITUDE'].notnull()].apply(lambda x: str(x["LATITUDE"]) + ", " + str(x["LONGITUDE"]), axis=1).to_list()
#removing duplicates
joined_column = list(set(joined_column))
joined_column = pd.DataFrame(list(set(joined_column)))


# Rename column 'A' to 'X'
joined_column.columns = ['Location']
joined_column

In [ ]:
from tqdm import tqdm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

# Initialize geolocator object
geolocator = Nominatim(user_agent='my_app', timeout=10)

revgeo = RateLimiter(geolocator.reverse, min_delay_seconds=0.001)

joined_column['address'] = joined_column['Location'].progress_apply(revgeo)
joined_column.head()


In [ ]:
from google.colab import files
# Export the DataFrame to a CSV file
joined_column.to_csv('joined_column.csv', index=False)
files.download('joined_column.csv')

In [ ]:
#getting the list of latitude and longitude for missing ONStreet values
on_street = df[df['ON STREET NAME'].isnull()& df['LATITUDE'].notnull()].apply(lambda x: str(x["LATITUDE"]) + ", " + str(x["LONGITUDE"]), axis=1).to_list()

#removing duplicates
on_street = list(set(on_street))

In [ ]:
# Remove values that are present in the 'joined_column Location' column
result = [value for value in on_street if value not in joined_column['Location'].tolist()]


On_street = pd.DataFrame(result)


# Rename column 'A' to 'X'
On_street.columns = ['Location']
On_street

In [ ]:
On_street['address'] = On_street['Location'].progress_apply(revgeo)
On_street.head()

100%|██████████| 16407/16407 [2:16:48<00:00,  2.00it/s]


,Location,address
0,"40.679825, -73.79451","(117-12, 146th Street, Queens, City of New Yor..."
1,"40.571987, -74.10823","(2460, Hylan Boulevard, New Dorp, Staten Islan..."
2,"40.879955, -73.87888","(Bainbridge Avenue & East 210th Street, Bainbr..."
3,"40.83087, -73.87495","(1245, Morrison Avenue, West Farms, Bronx Coun..."
4,"40.705135, -73.905334","(1911, Palmetto Street, Ridgewood, Queens, Cit..."


In [ ]:
# Export the DataFrame to a CSV file
On_street.to_csv('On_street.csv', index=False)
files.download('On_street.csv')

In [ ]:
missing_borough = pd.read_csv("/content/joined_column.csv")
missing_street = pd.read_csv("/content/On_street.csv")


# Join the dataframes based on the 'Location' column
missing_vals = pd.concat([missing_borough, missing_street]).drop_duplicates().reset_index(drop=True)


missing_vals

In [ ]:
#all boroughs in New york
bors = ['Bronx', 'Brooklyn', 'Queens', 'Manhattan', 'Staten Island']

bors

In [ ]:
missing_vals["address"] = missing_vals["address"].apply(lambda x: str(x))
# Split the 'address' column and extract the Borough, On street name and Zip code


# Define a function to get Boroughs if they match a value in the list
def replace_value(value):
    for i in bors:
      if i in value:
        return i
    else:
        return 'unknown'

# Apply the function to the address column
missing_vals['Borough'] = missing_vals['address'].apply(replace_value)

#On Street name
missing_vals["ON STREET"] = missing_vals["address"].apply(lambda x: str(x).split(", ")[1] if (x and ", " in x) else "")

#Zip Code
missing_vals["Zip Code"] = missing_vals["address"].apply(lambda x: str(x).split(", ")[-2] if (x and ", " in x) else "")

#Locatrion
missing_vals["LOCATION"] = missing_vals["Location"].apply(lambda x: "POINT (" + str(x).split(", ")[-1] + " " + str(x).split(", ")[0] + ")" if (x and ", " in x) else "" )

missing_vals.Borough.value_counts()

In [ ]:
#getting the number of records that have all vehicle type codes missing
df[df["VEHICLE TYPE CODE 1"].isna() & df["VEHICLE TYPE CODE 2"].isna() & df["VEHICLE TYPE CODE 3"].isna() & df["VEHICLE TYPE CODE 4"].isna() & df["VEHICLE TYPE CODE 5"].isna()].isna().sum()

In [ ]:
#deleting rocirds that have no vehicle type code
df = df.dropna(how='all', subset=["VEHICLE TYPE CODE 1", "VEHICLE TYPE CODE 2", 
                                  "VEHICLE TYPE CODE 3", "VEHICLE TYPE CODE 4", 
                                  "VEHICLE TYPE CODE 5",])


In [ ]:
# Merge the two DataFrames on the ID column and replace missing values in df1 with values from df2
df_merge = pd.merge(df, missing_vals, on='LOCATION', how='left')


df_merge['BOROUGH'].fillna(df_merge['Borough'], inplace=True)
df_merge['ZIP CODE'].fillna(df_merge['Zip Code'], inplace=True)
df_merge['ON STREET NAME'].fillna(df_merge['ON STREET'], inplace=True)

# The resulting DataFrame with replaced missing values
df_merge = df_merge.drop(['Location', 'address', "Borough", 'ON STREET', 'Zip Code'], axis=1)

In [ ]:
df_merge['BOROUGH'] = df_merge['BOROUGH'].str.upper()
df_merge['ON STREET NAME'] = df_merge['ON STREET NAME'].str.upper()

In [ ]:
df_merge

In [ ]:
from google.colab import files
# Export the DataFrame to a CSV file
df_merge.to_csv('Cleandata.csv', index=False)
files.download('Cleandata.csv')

In [ ]:
#importing data from  Analytics
df = pd.read_csv("/content/Cleandata.csv")

In [ ]:
df.columns

In [ ]:
#wrong values
mis_val = {'Sedan': 'Sedan',
 'Station Wagon/Sport Utility Vehicle': 'Station Wagon',
 'Taxi': 'Taxi',
 'Pick-up Truck': 'Truck',
 'Box Truck': 'Truck',
 'Bike': 'Bike',
 'Bus': 'Bus',
 'Motorcycle': 'Motorcycle',
 'Tractor Truck Diesel': 'Tractor',
 'Van': 'Van',
 'Ambulance': 'Ambulance',
 'E-Scooter': 'E-Scooter',
 'Convertible': 'Sedan',
 'Dump': 'Truck',
 'E-Bike': 'E-Bike',
 'PK': 'Truck',
 '4 dr sedan': 'Sedan',
 'Flat Bed': 'Truck',
 'Garbage Or Refuse': 'Garbage Truck',
 'Moped': 'Moped',
 'AMBULANCE': 'Ambulance',
 'Carry All': 'Utility Vehicle', 
 'Tow Truck / Wrecker': 'Tow Truck',
 'Chassis Cab': 'Truck',
 'Tractor Truck Gasoline': 'Tractor',
 'Tanker': 'Tanker',
 'Motorscooter': 'Moped',
 'Motorbike': 'Motorbike',
 'LIMO': 'Limousine',
 'Concrete Mixer': 'Construction Vehicle',
 'ambulance': 'Ambulance',
 'TRAILER': 'Trailer',
 '3-Door': 'Sedan',
 'FIRE TRUCK': 'Fire Truck',
 'Beverage Truck': 'Truck',
 'Flat Rack': 'Truck',
 'Lift Boom': 'Utility Vehicle',
 'School Bus': 'School Bus',
 'Refrigerated Van': 'Van',
 'Armored Truck': 'Armored Truck',
 'Multi-Wheeled Vehicle': 'Truck',
 '2 dr sedan': 'Sedan',
 'MOPED': 'Moped',
 'AMBU': 'Ambulance',
 'TRUCK': 'Truck',
 'Bulk Agriculture': 'Utility vehicle',
 'van': 'Van',
 'Minibike': 'Moped',
 'Stake or Rack': 'Trailer',
 'BOX TRUCK': 'Truck',
 'Trailer': 'Trailer',
 'FIRETRUCK': 'Fire Truck',
 'Open Body': 'Truck',
 'FDNY TRUCK': 'Fire Truck',
 'USPS': 'Van',
 'UTIL': 'Utility Vehicle',
 'REFG': 'Van',
 'moped': 'Moped',
 'FIRE ENGIN': 'Fire Truck',
 'Fire Truck': 'Fire Truck',
 'DELV': 'Van',
 'POSTAL TRU': 'Truck',
 'UNK': 'Unknown',
 'government': 'Government Vehicle',
 'postal tru': 'Truck',
 'Minicycle': 'Moped',
 'ambu': 'Ambulance',
 'DELIVERY': 'Van',
 'PICK UP': 'Truck',
 'FLATBED': 'Truck',
 'FDNY FIRE': 'Fire Truck',
 'TRK': 'Truck',
 'USPS TRUCK': 'Truck',
 'FDNY AMBUL': 'Ambulance',
 'DELIVERY V': 'Van',
 'Cargo Van': 'Van',
 'FDNY Ambul': 'Ambulance',
 'TOW TRUCK': 'Tow Truck',
 'SCOOTER': 'Scooter',
 'GARBAGE TR': 'Garbage Truck',
 'TRAC': 'Tractor',
 'Fdny ambul': 'Ambulance',
 'dump truck': 'Truck',
 'forklift': 'Forklift',
 'FORK LIFT': 'Forklift',
 'Pick up': 'Truck',
 'LIGHT TRAI': 'Trailer',
 'GOLF CART': 'Utility Vehicle',
 'Van Camper': 'Van',
 'UNKNOWN': 'Unknown',
 'Work van': 'Van',
 'Motorized Home': 'Van',
 'e skate bo': 'Skateboard',
 'Enclosed Body - Removable Enclosure': 'Truck',
 'MOTOR SCOO': 'Moped',
 'SUBURBAN': 'SUV',
 'Commercial': 'Commercial Vehicle',
 'E bike': 'E-Bike',
 'BACK HOE': 'Construction Vehicle',
 'fire truck': 'Fire Truck',
 'E-bike': 'E-Bike',
 'LCOMM': 'Unknown',
 'FREIG': 'Trailer',
 'PICK UP TR': 'Truck',
 'FDNY Fire': 'Fire Truck',
 'Ford Van': 'Van',
 'PICK-UP TR': 'Truck',
 'CHEVROLET': 'SUV',
 'Ford sprin': 'SUV',
 'hrse': 'Horse',
 'MAIL TRUCK': 'Truck',
 'EMS': 'Van',
 'COM TRANS': 'Commercial Vehicle',
 '0': 'Unknown',
 'E-BIKE': 'E-Bike',
 'DIRT BIKE': 'Motorbike',
 'pc': 'Sedan',
 'Hearse': 'Hearse',
 'Firetruck': 'Fire Truck',
 'FDNY EMT': 'Fire Truck',
 'OMR': 'Unknown',
 'FDNY FIRET': 'Fire Truck',
 'pick': 'Truck',
 'Tow': 'Tow Truck',
 'CARGO VAN': 'Van',
 'TRAIL': 'Trail',
 'DUMP': 'Truck',
 'Pickup with mounted Camper': 'Truck',
 'delviery': 'Van',
 'SCHOOL BUS': 'School Bus',
 'Tow Truck': 'Tow Truck',
 'AMB': 'Ambulance',
 'Horse Trai': 'Horse',
 'USPS #7530': 'Van',
 'FDNY': 'Fire Truck',
 'delivery v': 'Van',
 'E-Bik': 'E-Bike',
 'FDNY LADDE': 'Fire Truck',
 'FREIG DELV': 'Truck',
 'Utility.': 'Utility Vehicle',
 'PUSH SCOOT': 'Children Vehicle',
 'White ambu': 'Ambulance',
 'SELF INSUR': 'Unknown',
 'FREIGHTLIN': 'Truck',
 'FREIGHT TR': 'Truck',
 'Fire truck': 'Fire Truck',
 'USPS POSTA': 'Van',
 'Tow truck': 'Tow Truck',
 'escavator': 'Construction Vehicle',
 'P/SH': 'Unknown',
 'TRL': 'Trailer',
 'Lunch Wagon': 'Truck',
 'Dump truck': 'Truck',
 'ford econo': 'SUV',
 'commercial': 'Commercial Vehicle',
 'com': 'Commercial Vehicle',
 'tow truck': 'Tow Truck',
 'Go kart': 'Utility Vehicle',
 'FDNY #226': 'Fire Truck',
 'FREIGHT FL': 'Trailer',
 'Delv': 'Van',
 'truck': 'Truck',
 'Livestock Rack': 'Truck',
 'box truck': 'Truck',
 'USPS VAN': 'Van',
 '18 WHEELER': 'Trailer',
 'AMBULENCE': 'Ambulance',
 'FOOD TRUCK': 'Truck',
 'MOVING VAN': 'Van',
 'backhoe': 'Construction Vehicle',
 'COMMERCIAL': 'Commercial Vehicle',
 'Tractor': 'Tractor',
 'MOTORSCOOT': 'Moped',
 'cross': 'SUV',
 'JOHN DEERE': 'Utility Vehicle',
 'TRUCK VAN': 'Van',
 'Work Van': 'Van',
 'Lawnmower': 'Utility Vehicle',
 'SWT': 'Unknown',
 'COURIER': 'Van',
 'PSD': 'Sedan',
 'FDNY fire': 'Fire Truck',
 'dilevery t': 'Truck',
 'Front-Load': 'Trailer',
 'DRILL RIG': 'Construction Vehicle',
 'GEN  AMBUL': 'Ambulance',
 'Tractor tr': 'Tractor',
 'Pumper': 'Fire Truck',
 'OTH': 'Unknown',
 'TRUCK FLAT': 'Truck',
 'UNKN': 'Unknown',
 'UTILITY VE': 'Utility Vehicle',
 'suburban': 'SUV',
 'WORK VAN': 'Van',
 'CAT': 'Construction Vehicle',
 'Pick up Tr': 'Truck',
 'ELEC. UNIC': 'Unknown',
 'DELIVERY T': 'Truck',
 'MTA BUS': 'Bus',
 'MECHANICAL': 'Sedan',
 'NYC AMBULA': 'Ambulance',
 'ELECTRIC S': 'E-Scooter',
 'postal bus': 'van',
 'Trc': 'Truck',
 'NYC FD': 'Fire Truck',
 'Pedicab': 'Pedicab',
 'f-250': 'Truck',
 'FUSION': 'Sedan',
 'FDNY ENGIN': 'Fire Truck',
 'CHEVY EXPR': 'Van',
 'UT': 'Unknown',
 'Unknown': 'Unknown',
 'Bucket Tru': 'Truck',
 'usps': 'Van',
 'DOT EQUIPM': 'Truck',
 'ROAD SWEEP': 'Utility Vehicle',
 'Wh Ford co': 'Truck',
 'SLINGSHOT': 'Utility Vehicle',
 'Horse': 'Horse',
 'FD LADDER': 'Fire Truck',
 'Fork lift' : 'Forklift',
 'E REVEL SC': 'E-Scooter',
 'UPS TRUCK': 'Truck',
 'Freight': 'Truck',
 'BOBCAT FOR': 'Truck',
 'FDNY Engin': 'Fire Truck',
 'School bus': 'School Bus'}

vehicle_type_mapping = {
    'Sanitation': 'Truck',
    'Ice Cream': 'Truck',
    'Pickup': 'Truck',
    'Tractor Tr': 'Tractor',
    'Enclosed Body - Nonremovable Enclosure': 'Unknown',
    'Fdny Ems': 'Fire Truck',
    'Pallet': 'Unknown',
    'Lcom': 'Unknown',
    'Glnen': 'Sedan',
    'Sweeper': 'Road Sweeper',
    'Tl': 'Sedan',
    'Fd Truck': 'Fire Truck',
    'Fire': 'Fire Truck',
    'Itas': 'Sedan',
    'Uhaul Truc': 'Truck',
    '18 Weeler': 'Trailer',
    'Mailtruck': 'Truck',
    'E-Scoter': 'E-Scooter',
    'Lma': 'Unknown',
    'Skid Loade': 'Construction Vehicle',
    'T630 Forkl': 'Forklift',
    'Rds': 'Sedan',
    'Access A R': 'Bus',
    'Crane': 'Construction Vehicle',
    'Omt': 'Unknown',
    'Postal Veh': 'Truck',
    'Forklift T': 'Forklift',
    'City Mta B': 'Bus',
    'Pumper Tru': 'Fire Truck',
    'Trailor': 'Trailer',
    'Glben': 'Sedan',
    'Armored Tr': 'Armored Truck',
    'Sprinter V': 'Van',
    'Excavator': 'Construction Vehicle',
    'Pickup Tru': 'Truck',
    'Uhaul': 'Truck',
    'Cmix': 'Construction Vehicle',
    'Tract': 'Tractor',
    'Constructi': 'Construction Vehicle',
    'Flatbed Tr': 'Truck',
    'Tour Bus': 'Tour Bus',
    'Usps Mail': 'Van',
    'Us Postal': 'Van',
    'Jetski': 'JetSki',
    'Scoot': 'Scooter',
    'Power Shov': 'Construction Vehicle',
    'Ems Truck': 'Truck',
    'Nyc Bus': 'Bus',
    'E-Skateboa': 'Skateboard',
    'Special Pu': 'Special Purpose Vehicle',
    'Frieghtlin': 'Truck',
    'Ram': 'Truck',
    'Palfinger': 'Construction Vehicle',
    'Moped Scoo': 'Moped',
    'Electric M': 'E-bike',
    'Internatio': 'International',
    'Street Cle': 'Road Sweeper',
    'Mta Truck': 'Truck',
    'Escooter': 'E-Scooter',
    '4Dsd': 'Sedan',
    'Nyc Firetr': 'Fire Truck',
    'Street Swe': 'Road Sweeper',
    'Vanette': 'Van',
    'Ukn': 'Unknown',
    'Utility Tr': 'Utility Vehicle',
    'Mopd': 'Moped',
    'Tree Cutte': 'Tree Cutter',
    'Toyota': 'Sedan',
    'Bob Cat': 'Truck',
    'Escovator': 'Construction Vehicle',
    'Glp050Vxev': 'Forklift',
    'Van/Transi': 'Van',
    'Razor Scoo': 'E-Scooter',
    'Gas Scoote': 'Scooter',
    'Mobile': 'Unknown',
    'Haul For H': 'Van',
    'Pick Truck': 'Truck',
    'Commerical': 'Commercial Vehicle',
    'Glass Rack': 'Unknown',
    'Suv': 'SUV',
    'Vehicle 2': 'Unknown',
    'Tow Trk': 'Tow Truck'}


In [ ]:
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].str.title().replace(mis_val)
df['VEHICLE TYPE CODE 1'] = df['VEHICLE TYPE CODE 1'].str.title().replace(vehicle_type_mapping)
df['VEHICLE TYPE CODE 1'].value_counts()

Sedan                   2214
Station Wagon           1639
Truck                    143
Taxi                     121
Pick-Up Truck            109
Bike                     102
Motorcycle                51
Bus                       43
Tractor                   41
Van                       32
Ambulance                 23
E-Scooter                 21
E-Bike                    20
Moped                     12
Garbage Or Refuse          9
Utility Vehicle            8
Tow Truck                  5
Construction Vehicle       4
Unknown                    3
Fire Truck                 3
Tanker                     2
Trailer                    2
Armored Truck              1
Stake Or Rack              1
Name: VEHICLE TYPE CODE 1, dtype: int64

In [ ]:
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].str.upper().replace(mis_val)
df['VEHICLE TYPE CODE 2'] = df['VEHICLE TYPE CODE 2'].str.title().replace(vehicle_type_mapping)
df['VEHICLE TYPE CODE 2'].value_counts()

Sedan                   1389
Station Wagon            944
Bike                     258
Truck                    124
Pick-Up Truck             74
Motorcycle                58
Taxi                      45
E-Bike                    44
E-Scooter                 38
Tractor                   36
Van                       34
Bus                       28
Ambulance                 19
Moped                     12
Motorbike                 10
Garbage Or Refuse          8
Utility Vehicle            8
Fire Truck                 8
Tow Truck                  3
Scooter                    3
Tanker                     2
Road Sweeper               2
Forklift                   1
Construction Vehicle       1
Commercial Vehicle         1
Trailer                    1
Armored Truck              1
Limousine                  1
Skateboard                 1
Unknown                    1
International              1
Stake Or Rack              1
Pedicab                    1
Name: VEHICLE TYPE CODE 2, dtype: int64

In [ ]:
df['VEHICLE TYPE CODE 3'] = df['VEHICLE TYPE CODE 3'].str.title().replace(mis_val)
df['VEHICLE TYPE CODE 3'] = df['VEHICLE TYPE CODE 3'].str.title().replace(vehicle_type_mapping)
df['VEHICLE TYPE CODE 3'].value_counts()

Sedan            230
Station Wagon    196
Pick-Up Truck     12
Truck              7
Taxi               7
Bike               5
Motorcycle         2
Bus                2
Tractor            2
Van                1
Garbage Truck      1
Name: VEHICLE TYPE CODE 3, dtype: int64

In [ ]:
df['VEHICLE TYPE CODE 4'] = df['VEHICLE TYPE CODE 4'].str.title().replace(mis_val)
df['VEHICLE TYPE CODE 4'] = df['VEHICLE TYPE CODE 4'].str.title().replace(vehicle_type_mapping)
df['VEHICLE TYPE CODE 4'].value_counts()

Station Wagon    60
Sedan            58
Truck             4
Motorcycle        3
Pick-Up Truck     3
Bike              1
Van               1
Name: VEHICLE TYPE CODE 4, dtype: int64

In [ ]:
df['VEHICLE TYPE CODE 5'] = df['VEHICLE TYPE CODE 5'].str.title().replace(mis_val)
df['VEHICLE TYPE CODE 5'] = df['VEHICLE TYPE CODE 5'].str.title().replace(vehicle_type_mapping)
df['VEHICLE TYPE CODE 5'].value_counts()

Station Wagon    22
Sedan            21
Motorcycle        2
Pk                1
Name: VEHICLE TYPE CODE 5, dtype: int64

In [ ]:
df.BOROUGH = df.BOROUGH.str.title()
df['ON STREET NAME'] = df['ON STREET NAME'].str.title()

In [ ]:
from google.colab import files
# Export the DataFrame to a CSV file
df.to_csv('Clean.csv', index=False)
files.download('Clean.csv')